# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
# Read in the CSV file
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv", dtype="object", encoding="utf-8", usecols=range(1,10))

city_data_df.dropna(inplace = True) 
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Severo-Kuril'sk,50.68,156.12,50.38,86,99,17.85,RU,1601319875
1,Berezovyy,51.67,135.7,43.83,74,89,5.73,RU,1601319875
2,Fort Nelson,58.81,-122.7,46.4,81,75,2.35,CA,1601319876
3,Tiksi,71.69,128.87,30.88,93,0,20.31,RU,1601319876
4,Hobart,-42.88,147.33,46.4,87,90,3.6,AU,1601319713


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [9]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_data_df[["Lat", "Lng"]].astype(float)


# Convert Humidity to float and store
humidity = city_data_df["Humidity"].astype(float)

In [10]:
# Create a Heatmap layer
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=50, point_radius=4)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
